# NLP FROM SCRATCH: TRANSLATION WITH A SEQUENCE TO SEQUENCE NETWORK AND ATTENTION

This is the third and final tutorial on doing "NLP From Scratch", where we write our own classes and functions to preprocess the data to do our NLP modeling tasks. We hope after you complete this tutorial that you'll proceed to learn how _torchtext_ can handle much of this preprocessing for you in the three tutorials immediately following this one.

In this project we will be teaching a neural network to translate from French to English.

... to varying degrees of success.

This is made possible by the simple but powerful idea of the sequence to sequence network, in which two recurrent neural networks work together to transform one sequence to another. An encoder network condenses an input sequence into a vector, and a decoder network unfolds that vector into a new sequence.

To improve upon this model we'll use an attention mechanism, which lets the decoder learn to focus over a specific range of the input sequence.

In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import numpy as np

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Loading data files

The data for this project is a set of many thousands of English to French translation pairs.

In [ ]:
!wget https://download.pytorch.org/tutorial/data.zip
!unzip data.zip

Similar to the character encoding used in the character-level RNN tutorials, we will be representing each word in a language as a one-hot vector, or giant vector of zeros except for a single one (at the index of the word). Compared to the dozens of characters that might exist in a language, there are many many more words, so the encoding vector is much larger. We will however cheat a bit and trim the data to only use a few thousand words per language.

![diagram](https://pytorch.org/tutorials/_images/word-encoding.png)

We'll need a unique index per word to use as the inputs and targets of the networks later. To keep track of all this we will use a helper class called `Lang` which has word -> index (`word2index`) and index -> word (`index2word`) dictionaries, as well as a count of each word `word2count` to use to later replace rare words.

In [2]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = { 0: 'SOS', 1: 'EOS' }
        self.n_words = 2 # Count SOS and EOS
        
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
            
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [3]:
class Char:
    def __init__(self, name):
        self.name = name
        self.char2index = {}
        self.char2count = {}
        self.index2char = {}
        self.n_chars = 0
        
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
            
    def addWord(self, word):
        for char in word:
            self.addChar(char)
            
    def addChar(self, char):
        if char not in self.char2index:
            self.char2index[char] = self.n_chars
            self.char2count[char] = 1
            self.index2char[self.n_chars] = char
            self.n_chars += 1
        else:
            self.char2count[char] += 1

The files are all in Unicode, to simplify we will turn Unicode characters to ASCII, make everything lowercase, and trim most punctuation.

In [4]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [5]:
print(unicodeToAscii("À l'aide !"))
print(normalizeString("À l'aide !"))

print(normalizeString("Go."))
print(normalizeString("Va !"))

A l'aide !
a l aide !
go .
va !


To read the data file we will split the file into lines, and then split lines into pairs. The files are all English -> Other Language, so if we want to translate from Other Language -> English I added the `reverse` flag to reverse the pairs.

In [6]:
def readLangs(lang1, lang2, reverse=False): # e.g. lang1 = eng, lang2 = fra
    print("Reading lines...")
    
    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').read().strip().split('\n')
    
    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    
    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
        input_char = Char(lang2)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)
        input_char = Char(lang1)
        
    return input_lang, output_lang, input_char, pairs

Since there are a _lot_ of example sentences and we want to train something quickly, we'll trim the data set to only relatively short and simple sentence. Here the maximum length is 10 words (that includes ending punctuation) and we're filtering to sentences that translate to the form "I am" or "He is" etc. (accounting for apostrophes replaced earlier).

In [7]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p): # e.g. p = ['go.', 'va !']
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

The full process for preparing that data is:
- Read text file and split into lines, split lines into pairs
- Normalize text filter by length and content
- Make word lists from sentences in pairs

In [8]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, input_char, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words and chars...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
        input_char.addSentence(pair[0])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    print("Counted chars:")
    print(input_char.name, input_char.n_chars)
    
    return input_lang, output_lang, input_char, pairs

input_lang, output_lang, input_char, pairs = prepareData('eng', 'fra', True)

print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words and chars...
Counted words:
fra 4345
eng 2803
Counted chars:
fra 29
['elle est fille unique .', 'she s an only child .']


In [9]:
print(input_char.char2index)
print(input_char.char2count)
print(input_char.index2char)
print(input_char.n_chars)

{'j': 0, 'a': 1, 'i': 2, 'n': 3, 's': 4, '.': 5, 'e': 6, 'v': 7, 'b': 8, 'c': 9, 'u': 10, 'g': 11, 'r': 12, 'o': 13, 'f': 14, 'm': 15, 't': 16, 'h': 17, '!': 18, 'l': 19, 'd': 20, 'p': 21, 'z': 22, 'x': 23, 'y': 24, 'q': 25, 'k': 26, '?': 27, 'w': 28}
{'j': 4774, 'a': 13819, 'i': 15983, 'n': 15628, 's': 27771, '.': 10262, 'e': 40222, 'v': 4630, 'b': 1642, 'c': 5537, 'u': 16566, 'g': 1742, 'r': 12076, 'o': 13499, 'f': 2316, 'm': 6719, 't': 15608, 'h': 1468, '!': 197, 'l': 9856, 'd': 5313, 'p': 6098, 'z': 407, 'x': 747, 'y': 415, 'q': 1192, 'k': 37, '?': 145, 'w': 13}
{0: 'j', 1: 'a', 2: 'i', 3: 'n', 4: 's', 5: '.', 6: 'e', 7: 'v', 8: 'b', 9: 'c', 10: 'u', 11: 'g', 12: 'r', 13: 'o', 14: 'f', 15: 'm', 16: 't', 17: 'h', 18: '!', 19: 'l', 20: 'd', 21: 'p', 22: 'z', 23: 'x', 24: 'y', 25: 'q', 26: 'k', 27: '?', 28: 'w'}
29


### The CNN Model for Generating Character Embeddings

Consider the word 'cat', we pad it on both ends to get our maximum word length (this is mainly an implementation quirk since we can't have variable length layers at run time, our algorithm will ignore the pads).

We then apply a convolution layer on top that generates spatial coherence across characters, we use a maxpool to extract meaningful features out of our convolution layer. This now gives us a dense vector representation of each word. This representation will be concatenated with the Encoder output.

![cnn_model](https://raw.githubusercontent.com/TheAnig/NER-LSTM-CNN-Pytorch/master/images/cnn_model.png)

This snippet shows us how the CNN is implemented in PyTorch.

```
self.char_cnn3 = nn.Conv2d(in_channels=1, out_channels=self.out_channels, kernel_size=(3, char_embedding_dim), padding=(2, 0))
```

In [10]:
class CharacterCNN(nn.Module):
    def __init__(self, char_to_ix=None, char_embedding_dim=25, char_out_dim=25):
        """
        Input parameters:
            char_to_ix = Dictionary that maps characters to indices
            char_embedding_dim = Dimension of the character embeddings
            char_out_dim = Output dimension from the CNN encoder for character
        """
        super(CharacterCNN, self).__init__()
        self.char_embedding_dim = char_embedding_dim
        self.char_out_dim = char_out_dim
        
        # Initializing the character embedding layer
        self.char_embeds = nn.Embedding(len(char_to_ix), char_embedding_dim)
        self.init_embedding(self.char_embeds.weight)
        
        # Performing CNN encoding on the character embeddings
        self.char_cnn3 = nn.Conv2d(in_channels=1, out_channels=self.char_out_dim, kernel_size=(3, self.char_embedding_dim), padding=(2, 0))
        
    def forward(self, input):
        output = self.char_embeds(input).unsqueeze(1)
        output = self.char_cnn3(output).squeeze(-1).unsqueeze(1)
        output = F.max_pool2d(output, kernel_size=(1, output.size(-1))).squeeze(1).squeeze(-1)
        return output
        
    def init_embedding(self, input_embedding):
        """
        Initialize embedding
        """
        bias = np.sqrt(3.0 / input_embedding.size(1))
        nn.init.uniform_(input_embedding, -bias, bias)

In [11]:
model = CharacterCNN(input_char.char2index)

In [50]:
# input = torch.LongTensor([[0, 1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 5, 6, 7]])
input = torch.LongTensor([[6, 19, 19, 6, 6, 4, 16, 11, 6, 3, 16, 2, 19, 19, 6, 5]])
output = model(input)
print(output.shape)
print(output)

torch.Size([1, 25])
tensor([[0.0553, 0.1114, 0.0659, 0.1864, 0.0206, 0.3178, 0.0591, 0.1920, 0.0810,
         0.2072, 0.1618, 0.1455, 0.3715, 0.2233, 0.3685, 0.2558, 0.1612, 0.1304,
         0.1477, 0.3009, 0.2274, 0.3264, 0.3891, 0.1872, 0.1992]],
       grad_fn=<SqueezeBackward1>)


### The Seq2Seq Model

A Recurrent Neural Network, or RNN, is a network that operates on a sequence and uses its own output as input for subsequent steps.

A Sequence to Sequence network, or seq2seq network, or Encoder Decoder network, is a model consisting of two RNNs called the encoder and decoder. The encoder reads an input sequence and outputs a single vector, and the decoder reads that vector to produce an output sequence.

![diagram](https://pytorch.org/tutorials/_images/seq2seq.png)

Unlike sequence prediction with a single RNN, where every input corresponds to an output, the seq2seq model frees us from sequnce length and order, which makes it ideal for translation between two languages.

Consider the sentence "Je ne suis pas le chat noir" -> "I am not the black cat". Most of the words in the input sentence have a direct translation in the output sentence, but are in slightly different orders, e.g. "chat noir" and "black cat". Because of the "ne/pas" construction there is also one more word in the input sentence. It would be difficult to produce a correct translation directly from the sequence of input words.

With a seq2seq model the encoder creates a single vector which, in the ideal case, encodes the "meaning" of the input sentence into a single vector - a single point in some N dimensional space of sentences.

### The Encoder

The encoder of a seq2seq network is a RNN that outputs some value for every __character__ from the input sentence. For every input __character__ the encoder outputs a vector and a hidden state, and uses the hidden state for the next input word.

![diagram](https://pytorch.org/tutorials/_images/encoder-network.png)

In [14]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        
    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

### The Decoder

The decoder is another RNN that takes the encoder output vector(s) and outputs a sequence of words to create the translation.

#### Simple Decoder

In the simplest seq2seq decoder we use only last output of the encoder. This last output is sometimes called the _context_ vector as it encodes context from the entire sequence. This context vector is used as the initial hidden state of the decoder.

At every step of decoding, the decoder is given an input token and hidden state. The initial input token is the start-of-string `<SOS>` token, and the first hidden state is the context vector (the encoder's last hidden state).

![diagram](https://pytorch.org/tutorials/_images/decoder-network.png)

In [15]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

### Attention Decoder

If only the context vector is passed between the encoder and decoder, that single vector carries the burden of encoding the entire sequence.

Attention allows the decoder network to "focus" on a different part of the encoder's outputs for every step of the decoder's own outputs. First we calculate a set of _attention_ weights. These will be multiplied by the encoder output vectors to create a weighted combination. The result (called `attn_applied` in the code) should contain information about that specific part of the input sequence, and thus help the decoder choose the right output words.

![diagram](https://i.imgur.com/1152PYf.png)

Calculating the attention weights is done with another feed-forward layer `attn`, using the decoder's input and hidden state as inputs. Because there are sentences of all sizes in the training data, to actually create and train this layer we have to choose a maximum sentence length (input length, for encoder outputs) that it can apply to. Sentences of the maximum length will use all the attention weights, while shorter sentences will only use the first few.

![diagram](https://pytorch.org/tutorials/_images/attention-decoder-network.png)

In [ ]:
hidden_size = 256
char_out_dimension = 25
encoder1 = EncoderRNN(input_lang.n_words, hidden_size - char_out_dimension).to(device)
char_cnn3 = CharacterCNN(input_char.char2index, char_out_dim=char_out_dimension)
# attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words)

trainIters(encoder1, char_cnn3, attn_decoder1, 75000, print_every=5000)

In [16]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length
        
        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)
        
    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)
        
        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1
        )
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                encoder_outputs.unsqueeze(0))
        
        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)
        
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        
        output = F.log_softmax(self.out(output[0]), dim=1)
        
        return output, hidden, attn_weights
    
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

### Training

### Preparing Training Data

To train, for each pair we will need an input tensor (indexes of the words in the input sentence) and target tensor (indexes of the words in the target sentence). While creating these vectors we will append the EOS token to both sequences.

In [42]:
def wordIndexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def charIndexesFromSentence(char, sentence):
    return [char.char2index[character] for word in sentence.split(' ') for character in word]

def tensorFromSentence(lang, sentence):
    indexes = wordIndexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [43]:
sentence = 'je t aime'
print(sentence)
print(charIndexesFromSentence(input_char, sentence))
print(input_char.char2index)

je t aime
[0, 6, 16, 1, 2, 15, 6]
{'j': 0, 'a': 1, 'i': 2, 'n': 3, 's': 4, '.': 5, 'e': 6, 'v': 7, 'b': 8, 'c': 9, 'u': 10, 'g': 11, 'r': 12, 'o': 13, 'f': 14, 'm': 15, 't': 16, 'h': 17, '!': 18, 'l': 19, 'd': 20, 'p': 21, 'z': 22, 'x': 23, 'y': 24, 'q': 25, 'k': 26, '?': 27, 'w': 28}


### Training the Model

To train we run the input sentence through the encoder, and keep track of every output and the latest hidden state. Then the decoder is given the `<SOS>` token as its first input, and the last hidden state of the encoder as its first hidden state.

"Teacher forcing" is the concept of using the real target outputs as each next input, instead of using the decoder's guess as the next input. Using teacher forcing causes it to converge faster but when the trained network is exploited, it may exhibit instability.

You can observe outputs of teacher-forced networks that read with coherent grammar but wandar far from the correct translation - intuitively it has learned to represent the output grammar and can "pick up" the meaning once the teacher tells it the first few words, but it has not properly learned how to create the sentence from the translation in the first place.

Because if the freedom PyTorch's autograd gives us, we can randomly choose to use teacher forcing or not with a simple if statement. Turn `teacher_forcing_ratio` up to use more of it.

In [60]:
hidden_size = 256
char_out_dimension = 25
encoder1 = EncoderRNN(input_lang.n_words, hidden_size - char_out_dimension).to(device)
char_cnn3 = CharacterCNN(input_char.char2index, char_out_dim=char_out_dimension)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words)

trainIters(encoder1, char_cnn3, decoder1, 75000, print_every=5000)

3m 43s (- 52m 5s) (5000 6%) 2.9384
8m 2s (- 52m 16s) (10000 13%) 2.3753
12m 18s (- 49m 14s) (15000 20%) 2.0755


KeyboardInterrupt: 

In [59]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, input_char_indexes, encoder, char_cnn3, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    
    char_cnn3_outputs = torch.zeros(max_length, char_cnn3.char_out_dim, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]
        
#     print(input_char_indexes)
    
    char_cnn3_outputs = char_cnn3(torch.tensor([input_char_indexes], device=device)).unsqueeze(0)
        
    decoder_input = torch.tensor([[SOS_token]], device=device)

#     decoder_hidden = encoder_hidden
    decoder_hidden = torch.cat((char_cnn3_outputs, encoder_hidden), dim=2)
    
#     print(char_cnn3_outputs.shape)
#     print(encoder_outputs.shape)
#     print(encoder_hidden.shape)
#     print(decoder_hidden.shape)
#     print(decoder_input.shape)

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
#             decoder_output, decoder_hidden, decoder_attention = decoder(
#                 decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
#             decoder_output, decoder_hidden, decoder_attention = decoder(
#                 decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

This is a helper function to print time elapsed and estimated time remaining given the current time and progress %.

In [19]:
import time

def asMinutes(s):
    m = s // 60
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

The whole training process looks like this:
- Start a timer
- Initialize optimizers and criterion
- Create set of training pairs
- Start empty losses array for plotting

Then we call `train` many times and occasionally print the progress (% of examples, time so far, estimated time) and average loss.

In [20]:
def trainIters(encoder, char_cnn3, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    
#     training_pairs = [tensorsFromPair(random.choice(pairs))
#                       for i in range(n_iters)]
    training_pairs = []
    training_char_indexes = []
    for i in range(n_iters):
        pair = random.choice(pairs)
        training_pairs.append(tensorsFromPair(pair))
        training_char_indexes.append(charIndexesFromSentence(input_char, pair[0]))

    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]
        input_char_indexes = training_char_indexes[iter - 1]

        loss = train(input_tensor, target_tensor, input_char_indexes, encoder, char_cnn3,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

### Plotting results

Plotting is done with matplotlib, using the array of loss values `plot_losses` saved while training.

In [21]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locater puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

### Evaluation

Evaluation is mostly the same as training, but there are no targets so we simply feed the decoder’s predictions back to itself for each step. Every time it predicts a word we add it to the output string, and if it predicts the EOS token we stop there. We also store the decoder’s attention outputs for display later.

In [22]:
def evaluate(encoder, char_cnn, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()
        input_char_indexes = charIndexesFromSentence(input_char, sentence)

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [23]:
def evaluateRandomly(encoder, char_cnn, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
char_cnn3 = CharacterCNN(input_char.char2index)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, char_cnn3, attn_decoder1, 75000, print_every=5000)

In [ ]:
evaluateRandomly(encoder1, attn_decoder1)

In [ ]:
input = "je t aime !"
output_words, attentions = evaluate(encoder1, attn_decoder1, input)
output_sentence = ' '.join(output_words)
print('<', output_sentence)